In [1]:
import numpy as np
from pathlib import Path
import yaml
import cv2 as cv
from sklearn.model_selection import train_test_split

In [2]:
def prev_im(image, string=""):
    """
    Display an image in a window with OpenCV.

    Parameters:
    ---
    image : numpy.ndarray
        The image to be displayed, expected to be in the format readable by OpenCV (usually uint8, with dimensions [height, width, channels]).
    string : str, optional
        The title of the window in which the image will be displayed. Default is an empty string.

    Returns:
    ---
    None
    """
    cv.imshow(string, image)  # Display the image in a window with the title provided in 'string'.
    cv.waitKey(0)            # Wait indefinitely until a key is pressed.
    cv.destroyAllWindows()   # Close all OpenCV windows.

In [3]:
# Image Processing - Grayscale Conversion
def gen_gray_img(map_data: np.ndarray, debug=False):
    """
    Generate a gray-scale image from wafer data.

    Parameters:
    ---
    map_data : numpy.ndarray
        Wafer data map with pixel values of (0, 1, 2); 0=non-wafer, 1=good wafer, 2=bad wafer.
    debug : bool, optional
        If True, performs checks and logs issues instead of raising exceptions.

    Returns:
    ---
    numpy.ndarray
        Gray-scale image with pixel values (0, 128, 255).
    """
    expected_values = {0, 1, 2}
    unique_values = np.unique(map_data)
    
    if not expected_values.issuperset(unique_values):
        if debug:
            print(f"Unexpected values found in map_data: {unique_values}")
            return None
        else:
            raise ValueError(f"Map values do not match expected values (0, 1, 2)")

    gray = np.copy(map_data)
    gray[gray == 1] += 127
    gray[gray == 2] += 253  # direct assignment to max value for clarity

    return gray.astype(np.uint8)

In [4]:
# Image Processing - Colour Encoding
def gray_to_color(gray_map:np.ndarray,
                  bad_clr:tuple=(255,255,0),
                  good_clr:tuple=(25,102,255)):
    """
    Usage
    ---
    Convert gray-scale image to color image using `good_clr` and `bad_clr` BGR colors

    Parameters
    ---
    gray_map : ``numpy.ndarray``
    bad_clr : ``tuple`` optional,
        BGR color values to use for 'bad' pixels, `default=(255,255,0)` "Pumpkin"
    good_clr : ``tuple`` optional,
        BGR color values to use for 'good' pixels, `default=(25,102,255)` "Aqua"

    Returns
    ---
    BGR color image of wafer map, using `good_clr` and `bad_clr` pixel values

    """
    assert all([n in np.unique(gray_map) for n in [0,128,255]]), f"Gray scale values do not match expected values (0, 128, 255)"

    color_map = cv.cvtColor(np.copy(gray_map),cv.COLOR_GRAY2BGR) # B, G, R
    
    for d in range(color_map.shape[-1]):
            color_map[:,:,d][color_map[:,:,d] == 255] = bad_clr[d]
            color_map[:,:,d][color_map[:,:,d] == 128] = good_clr[d]
    
    return color_map

In [5]:
# Data Loading
import pandas as pd
data = np.load("/kaggle/input/mixedtype-wafer-defect-datasets/Wafer_Map_Datasets.npz")

In [6]:
# Seperate wafer map data and labels
imgs = data['arr_0'] # shape (38015, 52, 52)
lbls = data['arr_1'] # shape (38015, 8)

# Get unique labels
unique_lbls = np.unique(lbls,axis=0)

In [7]:
# Convert each label row to a tuple for it to be hashable
label_tuples = [tuple(label) for label in lbls]

# Use a dictionary to count occurrences of each unique label
label_count = {}
for label in label_tuples:
    if label in label_count:
        label_count[label] += 1
    else:
        label_count[label] = 1

# Display the counts
for label, count in label_count.items():
    print(f"Label {label}: {count} images")

Label (1, 0, 1, 0, 0, 0, 1, 0): 2000 images
Label (1, 0, 1, 0, 0, 0, 0, 0): 1000 images
Label (1, 0, 0, 1, 0, 0, 1, 0): 1000 images
Label (1, 0, 0, 1, 0, 0, 0, 0): 1000 images
Label (1, 0, 1, 0, 1, 0, 1, 0): 1000 images
Label (1, 0, 1, 0, 1, 0, 0, 0): 1000 images
Label (1, 0, 0, 1, 1, 0, 1, 0): 1000 images
Label (1, 0, 0, 1, 1, 0, 0, 0): 1000 images
Label (1, 0, 0, 0, 1, 0, 1, 0): 1000 images
Label (1, 0, 0, 0, 1, 0, 0, 0): 1000 images
Label (1, 0, 0, 0, 0, 0, 1, 0): 1000 images
Label (1, 0, 0, 0, 0, 0, 0, 0): 1000 images
Label (0, 1, 1, 0, 0, 0, 1, 0): 1000 images
Label (0, 1, 1, 0, 0, 0, 0, 0): 1000 images
Label (0, 1, 0, 1, 0, 0, 1, 0): 1000 images
Label (0, 1, 0, 1, 0, 0, 0, 0): 1000 images
Label (0, 1, 1, 0, 1, 0, 1, 0): 1000 images
Label (0, 1, 1, 0, 1, 0, 0, 0): 1000 images
Label (0, 1, 0, 1, 1, 0, 1, 0): 1000 images
Label (0, 1, 0, 1, 1, 0, 0, 0): 1000 images
Label (0, 1, 0, 0, 1, 0, 1, 0): 1000 images
Label (0, 1, 0, 0, 1, 0, 0, 0): 1000 images
Label (0, 1, 0, 0, 0, 0, 1, 0): 

In [8]:
# Load encoding file and create string labels
# YAML file provides a structured way to map arrays of numeric labels to human-readable strings, which describe various types of wafer defects.
encodes_path = "/kaggle/input/encodings-yaml/encodings.yaml"
with open(encodes_path, 'r') as enc:
    encd = yaml.safe_load(enc)

str_lbls = [str(l) for l in lbls]

In [9]:
# Show the number of unique pixel values in the uncleaned data
unique_pixels = set()  # To store all unique pixel values across all images
bad_wafers = []        # To store images that are considered bad based on pixel values

# Analyze each wafer image
for img in imgs:
    vals = np.unique(img).tolist()  # Get unique pixel values in the current image
    unique_pixels.update(vals)      # Update the set of unique pixel values

    # Check for unexpected pixel values
    if len(vals) > 3:
        bad_wafers.append(img)      # Append to bad wafers if unexpected values are found

# Display the results
print(f"Unique pixel values before cleaning: {unique_pixels}")
print(f"Number of bad wafers: {len(bad_wafers)}")

Unique pixel values before cleaning: {0, 1, 2, 3}
Number of bad wafers: 105


In [10]:
# Fix imgs with more than 3 pixel values
# should ONLY have 0,1,2 for values
# see https://github.com/Junliangwangdhu/WaferMap/issues/2
for im in imgs:
    val = np.unique(im)
    if len(val) > 3:
        im[im == 3] = 2

In [11]:
# Train-Test split
# y is generated by mapping the labels (lbls) to their corresponding encoded keys using a dictionary (encd).
X, y = imgs, [list(encd.keys())[list(encd.values()).index(k.tolist())] for k in lbls] 

# Split data into test and temp sets
X_train_temp, X_test, y_train_temp, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

# Further split the temp set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.2, random_state=10)

In [12]:
# Review counts to analyze the distribution of labels across your training, validation, and test sets after the data split.
train_lbl_counts = {k:v for k,v in zip(*np.unique(y_train,return_counts=True))}
val_lbl_counts = {k:v for k,v in zip(*np.unique(y_val,return_counts=True))}
test_lbl_counts = {k:v for k,v in zip(*np.unique(y_test,return_counts=True))}

In [13]:
import os

# Define directory paths
train_dir = '/kaggle/working/data/train'
val_dir = '/kaggle/working/data/val'
test_dir = '/kaggle/working/data/test'

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [14]:
# Train
# Directory setup for each class, image processing, image resizing, saving the processed image
for ct, t in enumerate(X_train):
    cls_dir = f"{train_dir}/{y_train[ct]}"
    if not Path(cls_dir).exists():
        _ = Path.mkdir(Path(cls_dir))
    else:
        pass
    filename = f"{cls_dir}/{ct}.png"
    color = gray_to_color(gen_gray_img(t))
    stride_szd = cv.resize(np.copy(color),(64,64),interpolation=cv.INTER_CUBIC)
    _ = cv.imwrite(filename,stride_szd)

In [15]:
# Validate
# Directory setup for each class, image processing, image resizing, saving the processed image
for cval,val in enumerate(X_val):
    cls_dir = f"{val_dir}/{y_val[cval]}"
    if not Path(cls_dir).exists():
        _ = Path.mkdir(Path(cls_dir))
    else:
        pass
    filename = f"{cls_dir}/{cval}.png"
    color = gray_to_color(gen_gray_img(val))
    stride_szd = cv.resize(np.copy(color),(64,64),interpolation=cv.INTER_CUBIC)
    _ = cv.imwrite(filename,stride_szd)

In [16]:
# Test
# Directory setup for each class, image processing, image resizing, saving the processed image
for ctest,test in enumerate(X_test):
    cls_dir = f"{test_dir}/{y_test[ctest]}"
    if not Path(cls_dir).exists():
        _ = Path.mkdir(Path(cls_dir))
    else:
        pass
    filename = f"{cls_dir}/{ctest}.png"
    color = gray_to_color(gen_gray_img(test))
    stride_szd = cv.resize(np.copy(color),(64,64),interpolation=cv.INTER_CUBIC)
    _ = cv.imwrite(filename,stride_szd)

In [17]:
# Generate color images for ALL wafer maps and save images to labeled directory
for k, v in encd.items():
    # Define the directory path
    directory_path = f'./wafers/{k}'
    
    # Check if the parent directory exists, if not create it
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
    
    # Get indices of matching groups
    values, *_ = np.where(np.array(str_lbls) == str(v).replace(',', ''))

    for idx in values:
        # Create grayscale image
        gray = gen_gray_img(imgs[idx])

        # Convert to color
        color = gray_to_color(gray)

        # Resize for YOLO model, (64 x 64); multiple of model stride 32
        stride_szd = cv.resize(np.copy(color), (64, 64), interpolation=cv.INTER_CUBIC)
        _ = cv.imwrite(f'./wafers/{k}/' + str(idx) + '.png', stride_szd)

In [18]:
!pip install ultralytics
from ultralytics import YOLO
from pathlib import Path
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 26.9 MB/s eta 0:00:00


In [19]:
# Load a model
model = YOLO('yolov8s-cls.yaml')  
data_dir = Path("/kaggle/working/data")

model.to('cuda') if model.device.type == 'cpu' else None #moves the model to a CUDA-enabled GPU

YOLOv8s-cls summary: 99 layers, 6361736 parameters, 6361736 gradients, 13.6 GFLOPs


YOLO(
  (model): ClassificationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()


In [20]:
print(os.getcwd())

/kaggle/working


In [21]:
# Train the model
model.train(data=data_dir,
            epochs=150,
            batch=16,
            imgsz=64,
            device=0,
            workers=12,
            project='wafer_defects',
            name='EXP00009',
            seed=10,
            deterministic=True,
            val=True,
            # save_json=True,
            # save_conf=True,
            dropout=0.15,   # default 0.0
            mosaic=0.96,    # default 1.0
            # flipud=0.0,     # default 0.0
            # fliplr=0.5,     # default 0.5
            # scale=0.5,      # default 0.5
            # translate=0.1,  # default 0.1
            degrees=50,     # default 0.0
            # mixup=0.0,      # default 0.0
            # copy_paste=0.0  # default 0.0
            patience=50,
            pretrained=False,
            optimizer='SGD',
            close_mosaic=0,
            plots=True
            )

Ultralytics YOLOv8.2.28 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.yaml, data=/kaggle/working/data, epochs=150, time=None, patience=50, batch=16, imgsz=64, save=True, save_period=-1, cache=False, device=0, workers=12, project=wafer_defects, name=EXP00009, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=10, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.15, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

2024-06-04 17:04:55,691	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-04 17:04:56,463	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=1000 with nc=38

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralyt

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


...

                   all      0.983          1

150 epochs completed in 1.643 hours.
Optimizer stripped from wafer_defects/EXP00009/weights/last.pt, 10.4MB
Optimizer stripped from wafer_defects/EXP00009/weights/best.pt, 10.4MB

Validating wafer_defects/EXP00009/weights/best.pt...
Ultralytics YOLOv8.2.28 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
YOLOv8s-cls summary (fused): 73 layers, 5123878 parameters, 0 gradients, 12.5 GFLOPs
train: /kaggle/working/data/train... found 21288 images in 38 classes ✅ 
val: /kaggle/working/data/val... found 5322 images in 38 classes ✅ 
test: /kaggle/working/data/test... found 11405 images in 38 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 167/167 [00:02<00:00, 78.92it/s]


                   all      0.983          1
Speed: 0.0ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to wafer_defects/EXP00009
Results saved to wafer_defects/EXP00009


lr/pg0,█▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▆███▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▆███▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/accuracy_top1,▁▄▇▇▇███████████████████████████████████
metrics/accuracy_top5,▁▆██████████████████████████████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/loss,█▅▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/loss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg0,0.00017


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x795bc437c610>
curves: []
curves_results: []
fitness: 0.9913566410541534
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9827132821083069, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9913566410541534}
save_dir: PosixPath('wafer_defects/EXP00009')
speed: {'preprocess': 0.025893637489797654, 'inference': 0.30716233395042836, 'loss': 0.000556130823180533, 'postprocess': 0.0005043435482009376}
task: 'classify'
top1: 0.9827132821083069
top5: 1.0

In [22]:
# Deploy model that has been previously trained
weights = "/kaggle/working/wafer_defects/EXP00009/weights/best.pt"
model = YOLO(weights)  

# Setting data directory
data_dir = Path("/kaggle/working/data")

# Evaluate the model on the test data split
model.val(data=data_dir,split='test',save_json=True,plots=True)

# Metric extraction
cls_count = model.metrics.confusion_matrix.nc
cls_names = model.names
matrix = model.metrics.confusion_matrix.matrix 

tp, fp = model.metrics.confusion_matrix.tp_fp() # true positive and false positive (total of true class incorrect)
fn = np.array([(matrix[:,c:c+1].sum() - matrix[c,c]) for c in range(cls_count)]) # false negative, (total of predicted class incorrect)

# Per-class metric calculation
class_metrics = dict()
all_p, all_r, all_f1, all_acc = [], [], [], []
for c in range(cls_count):
    recall = tp[c] / (tp[c] + fn[c])
    precision = tp[c] / (tp[c] + fp[c])
    accuracy = (tp[c] + 0) / (tp[c] + 0 + fp[c] + fn[c]) # zeros are True-Negatives, but there are none for this dataset
    f1_score = (2 * precision * recall) / (precision + recall)
    class_metrics[cls_names[c]] = np.array([precision, recall, f1_score, accuracy])
    all_p.append(precision)
    all_r.append(recall)
    all_f1.append(f1_score)
    all_acc.append(accuracy)

Ultralytics YOLOv8.2.28 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
YOLOv8s-cls summary (fused): 73 layers, 5123878 parameters, 0 gradients, 12.5 GFLOPs
train: /kaggle/working/data/train... found 21288 images in 38 classes ✅ 
val: /kaggle/working/data/val... found 5322 images in 38 classes ✅ 
test: /kaggle/working/data/test... found 11405 images in 38 classes ✅ 


test: Scanning /kaggle/working/data/test... 11405 images, 0 corrupt: 100%|██████████| 11405/11405 [00:03<00:00, 3416.83it/s]


test: New cache created: /kaggle/working/data/test.cache


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
               classes   top1_acc   top5_acc: 100%|██████████| 713/713 [00:05<00:00, 129.54it/s]


                   all      0.979          1
Speed: 0.0ms preprocess, 0.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [23]:
print(all_acc)

[0.9509803921568627, 0.95625, 0.9637561779242174, 0.9591836734693877, 0.9442622950819672, 0.9471947194719472, 0.9685534591194969, 0.9493243243243243, 0.919093851132686, 0.9614147909967846, 0.9523809523809523, 0.9795221843003413, 0.9661016949152542, 0.9935275080906149, 0.9244712990936556, 0.9840764331210191, 0.9907407407407407, 0.9376947040498442, 0.9686520376175548, 0.9329073482428115, 0.9503311258278145, 0.9413793103448276, 0.9672131147540983, 0.9688473520249221, 0.9646302250803859, 0.9479166666666666, 0.9695121951219512, 0.9577464788732394, 0.9290780141843972, 0.9619047619047619, 0.9611650485436893, 0.9725609756097561, 0.9478827361563518, 0.978021978021978, 0.8909090909090909, 0.9968454258675079, 0.9707112970711297, 0.9649122807017544]


In [24]:
print(np.mean(all_p))
print(np.mean(all_r))
print(np.mean(all_f1))
print(np.mean(all_acc))

0.977145690420757
0.9799956797141057
0.9782628667439056
0.957675175365652


In [26]:
# Saving evaluation results

# Define file paths
matrix_file = "/kaggle/working/runs/classify/val/confusion.csv"
metrics_file = "/kaggle/working/runs/classify/val/metrics.yaml"
names_file = "/kaggle/working/runs/classify/val/class_indices.yaml"
tp_fp_fn_file = "/kaggle/working/runs/classify/val/tp_fp_fn_counts.yaml"

# Save confusion matrix CSV file
import pandas as pd
_ = pd.DataFrame(matrix).to_csv(matrix_file)

# Save YAML files
metrics_export = {k: {'precision': float(v[0]), 'recall': float(v[1]), 'f1-score': float(v[2]), 'accuracy': float(v[3])} for k, v in class_metrics.items()}
metrics_export.update({'Average': {'precision': float(np.mean(all_p)), 'recall': float(np.mean(all_r)), 'f1-score': float(np.mean(all_f1)), 'accuracy': float(np.mean(all_acc))}})
with open(metrics_file, 'w') as met:
    yaml.safe_dump(metrics_export, met)

with open(names_file, 'w') as nf:
    yaml.safe_dump(cls_names, nf)

tp_fp_fn_counts = {int(i): {'tp': int(tp[i]), 'fp': int(fp[i]), 'fn': int(fn[i])} for i in range(cls_count)}
with open(tp_fp_fn_file, 'w') as cnt_f:
    yaml.safe_dump(tp_fp_fn_counts, cnt_f)